# Chapter IV-VII Heating and Cooling System - Hot Water Floor Heating / 暖冷房設備 - 温水暖房

## Maximum Heating Power / 最大暖房出力

The hourly maximum heating power $Q_{max,H,i}$ of the heating facility equipment etc. installed in the heating / cooling section $i$ is equal to the maximum heating power $Q_{max,H,rad,i}$ of radiator installed in the heating / cooling section $i$.

## Energy Consumption / エネルギー消費量

### 1. Power / 消費電力量

The hourly electric power $E_{E,H}$ is calculated by the equation below.  

$$
\large
E_{E,H} = E_{E,hs} + \sum_{i} (E_{E,rad,i})
$$

Where  
$E_{E,H}$ is the hourly electric power. / 1時間当たりの消費電力量(kWh/h)  
$E_{E,hs}$ is the hourly electric power of heat source machine for hot water heating. / 1時間当たりの温水暖房用熱源機の消費電力量(kWh/h)  
$E_{E,rad,i}$ is the hourly electric power of radiator installed in the heating / cooling section $i$. / 1時間当たりの暖冷房区画$i$に設置された放熱器の消費電力量(kWh/h)    

### 2. Kerosene Consumption / 灯油消費量 ★並び順

The hourly kerosene consumption $E_{K,H}$ is calculated by the equation below.

$$
\large
E_{K,H} = E_{K,hs}
$$

Where  
$E_{K,H}$ is the hourly kerosene consumption. / 1時間当たりの灯油消費量(MJ/h)  
$E_{K,hs}$ is the hourly kerosene consumption of heat source machine for hot water heating. / 1時間当たりの温水暖房用熱源機の灯油消費量(MJ/h)  

### 3. Gas Consumption / ガス消費量

The hourly gas consumption $E_{G,H}$ is calculated by the equation below.

$$
\large
E_{G,H} = E_{G,hs}
$$

Where  
$E_{G,H}$ is the hourly gas consumption. / 1時間当たりのガス消費量(MJ/h)  
$E_{G,hs}$ is the hourly gas consumption of heat source machine for hot water heating. / 1時間当たりの温水暖房用熱源機のガス消費量(MJ/h)  

### 4. Primary Energy Consumption by Other Fuels / その他の燃料による一次エネルギー消費量

The hourly primary energy consumption $E_{M,H}$ by other fuels is calculated by the equation below.

$$
\large
E_{M,H} = E_{M,hs}
$$

Where  
$E_{M,H}$ is the hourly primary energy consumption by other fuels. / 1時間当たりのその他の燃料による一次エネルギー消費量(MJ/h)  
$E_{M,hs}$ is the hourly primary energy consumption by other fuels of heat source machine for hot water heating. / 1時間当たりの温水暖房用熱源機のその他の燃料による一次エネルギー消費量(MJ/h)  

## Energy Consumption of Heat Source Machine for Hot Water Heating / 温水暖房用熱源機のエネルギー消費量

### 1. Energy Consumption / エネルギー消費量

The hourly electric power $E_{E,hs}$, the hourly kerosene consumption $E_{K,hs}$, the hourly gas consumption $E_{G,hs}$, the hourly primary energy consumption $E_{M,hs}$ by other fuels of heat source machine for hot water heating depends on the values listed in Table 3, and also according to the calculation methods listed in Table 3.  

<center>**Table 3(a). Calculation method for obtaining energy consumption of hot water heating source machine**</center>

|Types of heat source equipment for hot water heating|Method of calculation|
|-|-|
|Oil hot water heater<br>Oil hot water supply hot water heating machine|Appendix A|
|Gas hot water heater<br>Gas hot water supply hot water heating machine|Appendix B|
|Electric heater hot water heater<br>Electric heater hot water supply hot water heating machine|Appendix C|
|Electric heat pump hot water heating machine|Appendix D|
|Electric heat pump · gas instantaneous type combined use type hot water supply hot water heating machine<br>Hot water supply heat source: Gas instantaneous type<br>Heating heat source: Electric heat pump · gas instantaneous combination|Appendix E|
|Electric heat pump · gas instantaneous type combined use type hot water supply hot water heating machine<br>Hot water supply heat source:  Electric heat pump · gas instantaneous combination<br>Heating heat source: Gas instantaneous type|Appendix F|
|Electric heat pump · gas instantaneous type combined use type hot water supply hot water heating machine<br>Hot water supply heat source: Electric heat pump · gas instantaneous combination<br>Heating heat source: Electric heat pump · gas instantaneous combination|Appendix G|

<br>

<center>**Table 3(b). The dependent value for obtaining energy consumption of hot water heating source machine and the notation in the appendix**</center>

|Dependent value|Notation in calculation method (in appendix)|
|-|-|
|Outside air temperature $\theta_{ex}$|1 hour average outside air temperature $\theta_{ex}$|
|Outside air relative humidity $h_{ex}$|1 hour average outside air relative humidity $h_{ex}$|
|Hourly heating power $Q_{out,H,hs}$ of heat source machine for hot water heating|Hourly heating power $Q_{out,H,hs}$ of heat source machine for hot water heating|
|Hot water supply operation rate $r_{WS,hs}$ of heat source machine for hot water heating|Hourly average hot water supply operation rate $r_{WS,hs}$ of heat source machine for hot water heating|
|Outward hot water temperature $\theta_{SW,hs}$ of heat source machine for hot water heating|Hourly average outward hot water temperature $\theta_{SW,hs}$ of heat source machine for hot water heating|
|Return hot water temperature $\theta_{RW,hs}$ of heat source machine for hot water heating|Hourly average return hot water temperature $\theta_{RW,hs}$ of heat source machine for hot water heating|


### 2. Heating Power / 暖房出力

The hourly heating power $Q_{out,H,hs}$ of heat source machine for hot water heating is calculated by the equation below.

$$
\large
Q_{out,H,hs} = min(Q_{dmd,H,hs}, Q_{max,H,hs})
$$

Where  
$Q_{out,H,hs}$ is the hourly heating power of heat source machine for hot water heating. / 1時間当たりの温水暖房用熱源機の暖房出力(MJ/h)  
$Q_{max,H,hs}$ is the hourly maximum heating power of heat source machine for hot water heating. / 1時間当たりの温水暖房用熱源機の最大暖房出力(MJ/h)  
$Q_{dmd,H,hs}$ is the hourly hot water heat demand of heat source machine for hot water heating. / 1時間当たりの温水暖房用熱源機の温水熱需要(MJ/h)  

The hourly maximum heating power $Q_{max,H,hs}$ of heat source machine for hot water heating depends on the ambient temperature and humidity, and it depends on the calculation method listed in Table 3.

The hourly hot water heat demand $Q_{dmd,H,hs}$ of heat source machine for hot water heating is calculated by the equation below.

$$
\large
Q_{dmd,H,hs} = \sum_{i} (Q_{dmd,H,ln,i})
$$

Where  
$Q_{dmd,H,ln,i}$ is the hourly hot water heat demand of heat radiation system $i$. / 1時間当たりの放熱系統$i$の温水熱需要(MJ/h)  

### 3. Hot Water Supply Operation Rate / 温水供給運転率

The hot water supply operation rate $r_{WS,hs}$ of the heat source machine for hot water heating depends on the type of the heating / cooling section of the heat radiation system connected to the heat source machine for hot water heating and is calculated by the equation below.

When $Q_{dmd,H,hs} > 0$  
$$\large
r_{WS,hs} = \left\{ \begin{array}{ll}
    r_{WS,ln,i}&(放熱系統が1系統の場合)\\ 
    1&(放熱系統が2系統以上の場合)\\ 
    \end{array}
\right.
$$

<br>

When $Q_{dmd,H,hs} = 0$  
$$\large
r_{WS,hs} = 0
$$

Where  
$r_{WS,hs}$ is the hot water supply operation rate of heat source machine for hot water heating. / 温水暖房用熱源機の温水供給運転率  
$r_{WS,ln,i}$ is the hot water supply operation rate of heat radiation system $i$ (However, here $ i = 1 $.). / 放熱系統$i$の温水供給運転率（ただし、ここでは$i=1$である。）   

### 4. Outward Hot Water Temperature / 往き温水温度

Some of the heat source machine for hot water heating has a function to switch over the hot water temperature $\theta_{SW,hs}$ depending on the heating load of the heating and cooling section where the radiator is installed. 
This multiple candidate of hot water temperature is expressed as candidate $\theta_{SW,hs,op,p}$ and classification of the outward hot water temperature of the heat source machine for hot water heating. 
The division of the outward hot water temperature of theheat source machine for hot water heating is 3 at the maximum, and the number depends on the type and specification of the heat source machine for hot water heating. 
Table 4 shows the classification and candidates of the outward hot water temperature for each heat source machine for hot water heating.  

Where, 
$\theta_{SW,hs,op,1} > \theta_{SW,hs,op,2} > \theta_{SW,hs,op,3}$  
In this equation, $\theta_{SW,hs,op,p}$ represents the following numerical value.  
$\theta_{SW,hs,op,p}$ is the candidate of outward hot water temperature of heat source machine for hot water heating. / 温水暖房用熱源機の往き温水温度の候補$p$(℃)  

<center>**Table 4. the classification and candidates of the outward hot water temperature for each heat source machine for hot water heating (℃) **</center>

<table style="border: 1px solid #000;">
<thead>
<tr style="border: 1px solid #000;">
<th style="border: 1px solid #000;" colspan="2" rowspan="2">Types of heat source machine for hot water heating</th>
<th style="border: 1px solid #000;" colspan="3">Classification $p$ of candidates for outward hot water temperature</th>
</tr>
<tr>
<th style="border: 1px solid #000;">1</th>
<th style="border: 1px solid #000;">2</th>
<th style="border: 1px solid #000;">3</th>
</tr>
</thead>

<tbody>
<tr>
<th rowspan="2">Oil hot water heater, Oil hot water supply hot water heating machine</th>
<th>Conventional type</th>
<td>60</td>
<td>-</td>
<td>-</td>
</tr>
<tr>
<th>Latent heat recovery type</th>
<td>60</td>
<td>40</td>
<td>-</td>
</tr>

<tr>
<th rowspan="2">Gas hot water heater, Gas hot water supply hot water heating machine</th>
<th>Conventional type</th>
<td>60</td>
<td>-</td>
<td>-</td>
</tr>
<tr>
<th>Latent heat recovery type</th>
<td>60</td>
<td>40</td>
<td>-</td>
</tr>

<tr>
<th colspan="2">Electric heater hot water heater, Electric heater hot water supply hot water heating machine</th>
<td>60</td>
<td>-</td>
<td>-</td>
</tr>

<tr>
<th colspan="2">Electric heat pump hot water heating machine</th>
<td>55</td>
<td>45</td>
<td>35</td>
</tr>

<tr>
<th colspan="2">Electric heat pump · gas instantaneous type combined use type hot water supply hot water heating machine<br>
(Hot water supply heat source: Gas instantaneous type, <br>Heating heat source: Electric heat pump · gas instantaneous combination)</th>
<td>60</td>
<td>40</td>
<td>-</td>
</tr>

<tr>
<th colspan="2">Electric heat pump · gas instantaneous type combined use type hot water supply hot water heating machine<br>
(Hot water supply heat source: Electric heat pump · gas instantaneous combination, <br>Heating heat source: Gas instantaneous type)</th>
<td>60</td>
<td>40</td>
<td>-</td>
</tr>

<tr>
<th colspan="2">Electric heat pump · gas instantaneous type combined use type hot water supply hot water heating machine<br>
(Hot water supply heat source: Electric heat pump · gas instantaneous combination, <br>Heating heat source: Electric heat pump · gas instantaneous combination)</th>
<td>60</td>
<td>40</td>
<td>-</td>
</tr>

<tr>
<th colspan="2">Cogeneration system</th>
<td colspan="3">Depending on the type of cogeneration system</td>
</tr>

</tbody>
</table>


The (classification of) outward hot water temperature of the heat source machine for hot water heating shall be changed every hour and expressed as a classification $p_{hs}$ of the outward hot water temperature of the heat source machine for hot water heating. 
In the case of one heat dissipation system, the classification of the outward hot water temperature of the heat source machine for hot water heating shall be equal to the classification $p_{ln, i}$ of the required outward hot water temperature of the heat radiation system $i$ (where $i = 1$), and in the case of multiple heat dissipation systems, it is calculated by the equation below.

$$
\large
p_{hs} = min(p_{ln,1}, p_{ln,2}, … p_{ln,m})
$$

Where  
$p_{hs}$ is the classification of outward hot water temperature of heat source machine for hot water heating. / 温水暖房用熱源機の往き温水温度の区分(℃)  
$p_{ln,i}$ is the classification of the required outward hot water temperature of the heat radiation system $i$. / 放熱系統$i$の要求往き温水温度の区分(℃)  

### 5. Return Hot Water Temperature / 戻り温水温度

The return hot water temperature $\theta_{RW,hs}$ of heat source machine for hot water heating shall be calculated by the equation below within a range not exceeding the outword hot water temperature $\theta_{SW,hs}$ of the machine.

$$\large
\theta_{RW,hs} = \left\{ \begin{array}{ll}
    0.0301 \times (T_{dif})^2 - 0.1864 \times T_{dif} + 20&(熱源機の往き温水温度\theta_{SW,hs} = 60)\\ 
    0.0604 \times (T_{dif})^2 - 0.1881 \times T_{dif} + 20&(熱源機の往き温水温度\theta_{SW,hs} = 40)\\ 
    \end{array}
\right.
$$
<br><br>
$$\large
T_{dif} = \frac{\sum_{i}Q_{dmd,H,ln,i}}{\sum_{i}Q'_{max,rad,i} \times  0.027583}
$$
<br><br>
$$\large
Q'_{max,H,rad,i} = \left\{ \begin{array}{ll}
    Q_{max,H,rad,i}&(Q_{ln,dmd,i} > 0)\\ 
    0&(Q_{ln,dmd,i} = 0)\\ 
    \end{array}
\right.
$$

Where  
$T_{dif}$ is the logarithm average temperature difference. / 対数平均温度差(℃)  
$Q_{dmd,H,ln,i}$ is the hourly hot water heat demand of heat radiation system $i$. / 1時間当たりの放熱系統$i$の温水熱需要(MJ/h)  
$Q'_{max,H,rad,i}$ is the hourly maximum heating power of radiator installed in heating / cooling section $i$ when hot water heat demand occurs. / 1時間当たりの温水熱需要が発生する場合の暖冷房区画$i$に設置された放熱器の最大暖房出力(MJ/h)  
$Q_{max,H,rad,i}$ is the hourly maximum heating power of radiator installed in heating / cooling section $i$. / 1時間当たりの暖冷房区画$i$に設置された放熱器の最大暖房出力(MJ/h)  

## Heat Radiation System / 放熱系統

### 1. Hot Water Heat Demand / 温水熱需要

The hourly hot water heat demand $Q_{dmd,H,ln,i}$ of heat radiation system $i$ is calculated by the equation below.

$$
\large
Q_{dmd,H,ln,i} = Q_{dmd,H,rad,i} + Q_{loss,pp,i}
$$

Where  
$Q_{dmd,H,rad,i}$ is the hourly hot water heat demand of radiator installed in heating / cooling section $i$. / 1時間当たりの暖冷房区画$i$に設置された放熱器の温水熱需要(MJ/h)  
$Q_{loss,pp,i}$ is the hourly heat loss of piping $i$. / 1時間当たりの配管$i$の熱損失(MJ/h)  

### 2. Hot Water Supply Operation Rate / 温水供給運転率

The hot water supply operation rate $r_{WS,ln,i}$ of heat radiation system $i$ is calculated by the equation below.

$$
\large
r_{WS,ln,i} = r_{WS,rad,i}
$$

Where  
$r_{WS,rad,i}$ is the hot water supply operation rate of heat sink installed in heating / cooling section $i$. / 暖冷房区画$i$に設置された放熱器の温水供給運転率  

### 3. Classification of Requested Hot Water Temperature / 要求往き温水温度の区分

The method of determining the classification $P_{ln,i}$ of the requested outward hot water temperature of the heat radiation system $i$ is shown below.  
In Table 4, in the case of a hot water heating source machine having Category 3 of the requested outward hot water temperature, and when there is no unprocessed heating load in the heat radiation system $i$ at the outward hot water temperature of that classification, it is classified as Category 3. 
Next, in Table 4, 
in the case of a hot water heating source machine having Category 2 of the requested outward hot water temperature, and when there is no unprocessed heating load in the heat radiation system $i$, it is classified as Category 2. 
If it does not apply to either, it is classified as Category 1. 
Therefore, the following calculation is performed up to three times by changing the outward hot water temperature. 
Hereinafter, the outward hot water temperature to be set at that time is simply expressed as a outward hot water temperature $\theta_{SW}$.

## Piping / 配管

The hourly heat loss $Q_{loss,pp,i}$ of piping $i$ is calculated by the equation below.

$$\large
Q_{loss,pp,i} = ((\theta_{SW} - (\theta_{ex} \times 0.7 + 20 \times 0.3)) \times L_{pp,ex,i} + (\theta_{SW} - 20) \times L_{pp,in,i})\\
\large
\times K_{loss,pp,i} \times r_{WS,rad,i} \times 3600 \times 10^{-6}
$$

Where  
$\theta_{SW}$ is the outward hot water temperature. / 往き温水温度(℃)    
$\theta_{ex}$ is the outside air temperature. / 外気温度(℃)  
$L_{pp,ex,i}$ is the length of piping $i$ outside insulation compartment. / 配管$i$の断熱区画外における長さ(m)  
$L_{pp,in,i}$ is the length of piping $i$ inside insulation compartment. / 配管$i$の断熱区画内における長さ(m)  
$K_{loss,pp,i}$ is the line heat loss coefficient of piping $i$. / 配管$i$の線熱損失係数(W/mK)  
$r_{WS,rad,i}$ is the hot water supply operation rate of radiator installed in heating / cooling section $i$. / 暖冷房区画$i$に設置された放熱器の温水供給運転率  

The length $L_{pp,ex,i}$ of piping $i$ outside insulation compartment, the length $L_{pp,in,i}$ of piping $i$ inside insulation compartment and the line heat loss coefficient $K_{loss,pp,i}$ of piping $i$ are specified in Appendix I.

## Radiator / 放熱器

### 1. Supply Power / 供給熱量

The hourly hot water heat demand $Q_{dmd,H,rad,i}$ of radiator installed in heating / cooling section $i$ is calculated by the equation below.

$$\large
Q_{dmd,H,rad,i} = Q_{T,H,rad,i} + Q_{loss,rad,i}
$$

Where  
$Q_{T,H,rad,i}$ is the hourly process heating load of radiator installed in heating / cooling section $i$. / 1時間当たりの暖冷房区画$i$に設置された放熱器の処理暖房負荷(MJ/h)  
$Q_{loss,rad,i}$ is the hourly heat loss of radiator installed in heating / cooling section $i$. / 1時間当たりの暖冷房区画$i$に設置された放熱器の熱損失(MJ/h)  

### 2. Power / 消費電力量

The hourly electric energy consumption $E_{E,rad,i}$ of the radiator installed in the heating / cooling section $i$ shall be 0 when the type of radiator is a panel radiator or hot water floor heating. 
Furthermore, when the type of radiator is fan-convector, it depends on the hourly process heating load $Q_{T,H,rad,i}$ of radiator installed in the heating / cooling section $i$ and the outward hot water temperature $\theta_{SW}$. Its calculation method is defined in Appendix K.

### 3. Heat Loss / 熱損失

The hourly heat loss $Q_{loss,rad,i}$ of the radiator installed in the heating / cooling section $i$ shall be 0 when the type of radiator is a panel radiator or fan-convector. Furthermore, when the type of radiator is hot water floor heating, it depends on the hourly process heating load $Q_{T,H,rad,i}$ of radiator installed in the heating / cooling section $i$. Its calculation method is defined in Appendix L.

### 4. Hot Water Supply Operation Rate / 温水供給運転率

The hot water supply operation rate $r_{WS,rad,i}$ of the radiator installed in the heating / cooling section $i$ depends on the hourly process heating load $Q_{T,H,rad,i}$ of radiator installed in the heating / cooling section $i$.  
The calculation method is defined respectively, depending on the type of radiator:  
For a panel radiator, in Appendix J.  
For fan-convector, in Appendix K.  
For hot water floor heating, in Appendix L.

### 5. Process Heating Load / 処理暖房負荷

The hourly process heating load $Q_{T,H,rad,i}$ of the radiator installed in the heating / cooling section $i$ is equal to the process heating load $Q_{T,H,i}$ of the heating and cooling facility equipment etc. installed in the heating / cooling section $i$ described in Chapter 4  section 1.

### 6. Maximum Heating Power / 最大暖房出力

The hourly maximum heating power $Q_{max,H,rad,i}$ of the radiator installed in the heating / cooling section $i$ depends on the outward hot water temperature $\theta_{SW}$.  
The calculation method is defined respectively, depending on the type of radiator:  
For a panel radiator, in Appendix J.  
For fan-convector, in Appendix K.  
For hot water floor heating, in Appendix L.